In [1]:
import nltk.stem
import re
import numpy as np
import os
import nltk
import pandas as pd
from src.SemEvalData import SemEvalData
from src.JigsawData import JigsawData
from src.preprocessing import preprocess_bayes, getSpansByToxicWords, getToxicWordsBayes
import joblib
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import sklearn.feature_extraction.text
import sklearn.naive_bayes
import sklearn.metrics
import sklearn.model_selection
import sklearn.pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Julia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
#stemmer = nltk.stem.SnowballStemmer('english')
lemmer = nltk.stem.WordNetLemmatizer()

In [9]:
## load data
train_data_semeval = SemEvalData()
train_data_semeval.load_data("data/tsd_trial.csv")
train_df_preprocessed = train_data_semeval.preprocess()


In [16]:
i = 0
for v in train_df_preprocessed['toxic_words']:
    if( i > 90):
        break
    print(v)
    i = i+1

['moron', 'bigot']
['idiots']
['idiots']
['stupid']
[]
['idiot']
['o try to turn salem into some kind of new-stupid-fucking-asshole-topia. this will drive up the price of housing and displace  people down here. having lived in san clemente for a time myself i think youre dumb as shit to have moved up here. the fuck']
[]
['stupid']
['dont make ignorant statements']
['idiot']
['mexicans', 'rapists', 'drug dealers']
['psychopath']
[]
['dumb']
[]
['hypocrite']
['ridiculous']
['morons']
['hypocrisy', 'stupidity']
['stupid']
['idiot']
['r cho']
['stupid']
['nanakuli', 'ignorant']
['ridiculous']
['useless']
['illegal immigrants, crime , social parasites and welfare refugees', 'duterte is right killing druggies', 'islamism and a fascist dictator']
[]
['idiots']
['the sock puppet']
['idiot']
['put this pathetic rapists', 'let him rot with stale meals and a saggy cot']
[]
['liberalism leads to stupidity', 'laziness', 'largassitis']
['idiocy']
['stupid blind', 'white']
['stupid']
['idiot']
['sexu

In [5]:
train_data = {
    'sentence':  train_df_preprocessed.sentences.sum(),
    'toxicity_sentence': train_df_preprocessed.toxicity_sentence.sum()
        }

train_df = pd.DataFrame (train_data, columns = ['sentence','toxicity_sentence'])

In [6]:

train_df['sentence'] = train_df.apply(lambda row: preprocess_bayes(row.sentence), axis=1)
print(train_df.head(5))

                                            sentence  toxicity_sentence
0                    because hes a moron and a bigot                1.0
1             its not any more complicated than that                0.0
2  how about we stop protecting idiots and let na...                1.0
3  we can always submit their names for the darwi...                0.0
4  if people  were  smart they would  boycott thi...                1.0


In [7]:
def stemming(text):
    # Stem words
    #data[i] = ' '.join([stemmer.stem(word) for word in data[i].split()])
    data = ' '.join([lemmer.lemmatize(word) for word in text.split()])
    return data


In [8]:
# Perform a grid search to find the best hyperparameters
def grid_search(train):
    # Create a pipeline
    clf_pipeline = sklearn.pipeline.Pipeline([
        ('v', CountVectorizer(strip_accents='ascii', stop_words='english')),
        ('t', TfidfTransformer()), 
        ('c', sklearn.naive_bayes.MultinomialNB(fit_prior=True, class_prior=None))
        ])
    # Set parameters (name in pipeline + name of parameter)
    parameters = { 
        'v__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4)], 
        'v__lowercase': (True, False), 
        't__use_idf': (True, False), 
        'c__alpha': (0.3, 0.6, 1.0) }
    # Create a grid search classifier
    gs_classifier = sklearn.model_selection.GridSearchCV(clf_pipeline, parameters, cv=5, iid=False, n_jobs=2, scoring='accuracy', verbose=1)
    
    # Start a search (Warning: takes a long time if the whole dataset is used)
    # Slice: (train.data[:4000], train.target[:4000])
    gs_classifier = gs_classifier.fit(train.data, train.target)
    # Print results
    print('---- Results ----')
    print('Best score: ' + str(gs_classifier.best_score_))
    for name in sorted(parameters.keys()):
        print('{0}: {1}'.format(name, gs_classifier.best_params_[name]))

In [9]:
train_df['sentence'] = train_df.apply(lambda row: stemming(row.sentence), axis=1)

In [10]:
print(train_df.head())

                                            sentence  toxicity_sentence
0                     because he a moron and a bigot                1.0
1              it not any more complicated than that                0.0
2  how about we stop protecting idiot and let nat...                1.0
3  we can always submit their name for the darwin...                0.0
4  if people were smart they would boycott this i...                1.0


In [11]:
# Train and evaluate a model
def train_and_evaluate(train):
    
    # Convert to bag of words
    count_vect = CountVectorizer(strip_accents='ascii', stop_words='english', lowercase=True, ngram_range=(1,1))
    X = count_vect.fit_transform(train['sentence'])
    # Convert from occurrences to frequencies
    # Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.
    # To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies.
    transformer = TfidfTransformer()
    X = transformer.fit_transform(X)
    # Create a model
    # get the first vector out (for the first document) 
    first_vector_tfidfvectorizer=X[0] 
 
    # place tf-idf values in a pandas data frame 
    df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=count_vect.get_feature_names(), columns=["tfidf"]) 
    df.sort_values(by=["tfidf"],ascending=True)
    print(df.head)
    model = sklearn.naive_bayes.MultinomialNB(alpha=0.3, fit_prior=True, class_prior=None)
    # Train the model
    model.fit(X, train['toxicity_sentence'])
    # Save models
    joblib.dump(count_vect, 'vectorizer.jbl')
    joblib.dump(transformer, 'transformer.jbl')
    joblib.dump(model, 'model.jbl')
    # Evaluate on training data
    print('-- Training data --')
    predictions = model.predict(X)
    print('here',model.coef_, len(count_vect.get_feature_names()), predictions)
#     neg_class_prob_sorted = model.feature_log_prob_[0, :].argsort()
#     pos_class_prob_sorted = model.feature_log_prob_[1, :].argsort()

#     print(np.take(count_vect.get_feature_names(), neg_class_prob_sorted[:10]))
#     print(np.take(count_vect.get_feature_names(), pos_class_prob_sorted[:10]))
    print(predictions,X, train['sentence'])
    accuracy = sklearn.metrics.accuracy_score(train['toxicity_sentence'], predictions)
    print('Accuracy: {0:.2f}'.format(accuracy * 100.0))
    print('Classification Report:')
    print(sklearn.metrics.classification_report(train['toxicity_sentence'], predictions))
    print('')
#     Evaluate with 10-fold CV
    print('-- 10-fold CV --')
    predictions = sklearn.model_selection.cross_val_predict(model, X, train['toxicity_sentence'], cv=10)
    accuracy = sklearn.metrics.accuracy_score(train['toxicity_sentence'], predictions)
    print('Accuracy: {0:.2f}'.format(accuracy * 100.0))
    print('Classification Report:')
    print(sklearn.metrics.classification_report(train['toxicity_sentence'], predictions))
    return model


In [12]:

train_and_evaluate(train_df)

<bound method NDFrame.head of           tfidf
aa          0.0
ability     0.0
abject      0.0
able        0.0
abortion    0.0
...         ...
zion        0.0
zip         0.0
zombie      0.0
zone        0.0
zygote      0.0

[4125 rows x 1 columns]>
-- Training data --
here [[-9.22107255 -8.51797274 -9.22107255 ... -9.22107255 -9.22107255
  -9.22107255]] 4125 [1. 0. 1. ... 0. 0. 0.]
[1. 0. 1. ... 0. 0. 0.]   (0, 2419)	0.5829675030880128
  (0, 354)	0.8124954709678864
  (1, 725)	1.0
  (2, 3527)	0.2710466012552448
  (2, 2902)	0.3578505509617729
  (2, 2797)	0.37212675849741617
  (2, 2479)	0.3467770641029903
  (2, 2167)	0.2689345826546366
  (2, 1848)	0.1986127568357092
  (2, 1551)	0.3922479429093661
  (2, 379)	0.3922479429093661
  (2, 46)	0.3467770641029903
  (3, 3564)	0.6233413233842592
  (3, 900)	0.5686786636936917
  (3, 262)	0.5367030575850278
  (4, 3704)	0.3042103656034832
  (4, 3624)	0.23525102568681291
  (4, 3383)	0.5550665027433567
  (4, 3137)	0.3042103656034832
  (4, 3121)	0.288605254

MultinomialNB(alpha=0.3)

In [13]:
##load from file
count_vect = joblib.load('vectorizer.jbl')
transformer = joblib.load('transformer.jbl')
model = joblib.load('model.jbl')

In [28]:
samples_to_test = train_df.head(20)
for sample in samples_to_test.itertuples():
    print(sample)
    x = count_vect.transform([sample.sentence])
    y = model.predict(x)
    if y == 1.0:
        toxic = getToxicWordsBayes(count_vect, x, 0.5)
        spans = getSpansByToxicWords(toxic,sample.sentence)

Pandas(Index=0, sentence='because he a moron and a bigot', toxicity_sentence=1.0)
['bigot' 'moron'] because he a moron and a bigot
bigot
5
5
moron
5
5
[25, 26, 27, 28, 29, 13, 14, 15, 16, 17]
Pandas(Index=1, sentence='it not any more complicated than that', toxicity_sentence=0.0)
Pandas(Index=2, sentence='how about we stop protecting idiot and let nature add some bleach to the gene pool', toxicity_sentence=1.0)
['add' 'bleach' 'gene' 'idiot' 'let' 'nature' 'pool' 'protecting' 'stop'] how about we stop protecting idiot and let nature add some bleach to the gene pool
add
3
3
bleach
6
6
gene
4
4
idiot
5
5
let
3
3
nature
6
6
pool
4
4
protecting
10
10
stop
4
4
[50, 51, 52, 59, 60, 61, 62, 63, 64, 73, 74, 75, 76, 29, 30, 31, 32, 33, 39, 40, 41, 43, 44, 45, 46, 47, 48, 78, 79, 80, 81, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 13, 14, 15, 16]
Pandas(Index=3, sentence='we can always submit their name for the darwin award', toxicity_sentence=0.0)
Pandas(Index=4, sentence='if people were smart they